In [1]:
import os
import sys
sys.path.append("../..")
import phage_init

In [2]:
#load the saved matrices
import pickle
train_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"train_x_data2_full.p"), "rb" ) )
test_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"test_x_data2_full.p"), "rb" ) )
train_Y=pickle.load(open( os.path.join(phage_init.data_dir,"train_y_data2_full.p"), "rb" ) )
test_Y=pickle.load(open( os.path.join(phage_init.data_dir,"test_y_data2_full.p"), "rb" ) )
mean_total=pickle.load(open( os.path.join(phage_init.data_dir,"mean_data2_full.p"), "rb" ) )
std_total=pickle.load(open( os.path.join(phage_init.data_dir,"std_data2_full.p"), "rb" ) )

In [3]:
di_train     = train_X_total[:,0:400]
tri_train    = train_X_total[:,400:8400]
di_sc_train  = train_X_total[:,8400:8449] 
tri_sc_train = train_X_total[:,8449:8792]
tt_train     = train_X_total[:,8792:]

In [4]:
di_test     = test_X_total[:,0:400]
tri_test    = test_X_total[:,400:8400]
di_sc_test  = test_X_total[:,8400:8449] 
tri_sc_test = test_X_total[:,8449:8792]
tt_test     = test_X_total[:,8792:]

In [5]:
mean_di     = mean_total[0:400]
mean_tri    = mean_total[400:8400]
mean_di_sc  = mean_total[8400:8449] 
mean_tri_sc = mean_total[8449:8792]
mean_tt     = mean_total[8792:]

In [6]:
std_di     = std_total[0:400]
std_tri    = std_total[400:8400]
std_di_sc  = std_total[8400:8449] 
std_tri_sc = std_total[8449:8792]
std_tt     = std_total[8792:]

In [7]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [8]:
#print(di_train.shape)
print(tt_train.shape)
train_X =numpy.concatenate((tri_train,tt_train),axis=1)
test_X  =numpy.concatenate((tri_test,tt_test),axis=1)
#rain_X=tri_train
#test_X=tri_test
model_name='tri_p_data2.h5'
print(train_X.shape)
print(test_X.shape)

(50000, 8)
(50000, 8008)
(7774, 8008)


In [9]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13154055313012006638
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14246058882856067225
physical_device_desc: "device: XLA_CPU device"
]


In [10]:
print(train_X.shape[1])
f_num=train_X.shape[1]


8008


In [11]:
#model with 2 leyers of 100 LSTM neurons
model = Sequential()
opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.add(Dense(f_num, input_dim=f_num, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8008)              64136072  
_________________________________________________________________
dropout_1 (Dropout)          (None, 8008)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               1601800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total para

In [14]:
model.fit(train_X, train_Y, epochs=150,verbose=1, batch_size=2000)

Epoch 1/150
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0225 - acc: 0.9944
Epoch 2/150
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0223 - acc: 0.9944
Epoch 3/150
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0245 - acc: 0.9938
Epoch 4/150
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0229 - acc: 0.9937
Epoch 5/150
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0236 - acc: 0.9942
Epoch 6/150
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0244 - acc: 0.9937
Epoch 7/150
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0227 - acc: 0.9937
Epoch 8/150
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0232 - acc: 0.9941
Epoch 9/150
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0218 - acc: 0.9943
Epoch 10/150
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0222 - a

In [15]:
scores = model.evaluate(test_X, test_Y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

7774/7774 [==============================] - 9s 1ms/step
Accuracy: 90.87%


In [16]:
#you can save the as a file so you dont have to train it every time
model.save( os.path.join(phage_init.model_dir,model_name))

In [17]:
from sklearn.metrics import classification_report


test_Y_index = test_Y.argmax(axis=1) # Convert one-hot to index
test_Y_predicted = model.predict_classes(test_X)
labels_names=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
             "Tail shaft","Collar","Head-Tail joining"]
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
#print (test_Y_index)

                   precision    recall  f1-score   support

     Major capsid       0.94      0.95      0.94      1836
     Minor capsid       0.71      0.52      0.60       105
        Baseplate       0.93      0.84      0.88       308
       Major tail       0.84      0.92      0.88       423
       Minor tail       0.83      0.80      0.81       499
           Portal       0.93      0.96      0.94      2473
       Tail fiber       0.80      0.72      0.76       412
       Tail shaft       0.95      0.94      0.95       677
           Collar       0.74      0.79      0.76       149
Head-Tail joining       0.92      0.90      0.91       892

        micro avg       0.91      0.91      0.91      7774
        macro avg       0.86      0.83      0.84      7774
     weighted avg       0.91      0.91      0.91      7774



In [ ]:
from collections import Counter
zz=Counter(test_Y_index)
sample_w=[zz[i] for i in range(0,10,1)]
print(zz)
print(sample_w)
print()

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

CM=confusion_matrix(test_Y_index, test_Y_predicted)
CM_n=CM/numpy.array(sample_w)[:,None]
scale_up=1.3
plt.figure(figsize=[6.4*scale_up, 4.8*scale_up])
plt.imshow(CM_n, interpolation='nearest')
plt.title('CM')
plt.colorbar()
tick_marks = numpy.arange(len(labels_names))
plt.xticks(tick_marks, labels_names, rotation=90)
plt.yticks(tick_marks, labels_names)
fmt = '.2f'
for i, j in itertools.product(range(CM_n.shape[0]), range(CM_n.shape[1])):
        plt.text(j, i, format(CM_n[i, j], fmt),horizontalalignment="center",verticalalignment='center',
                color="white" if CM_n[i, j] < 0.25 else "black")
plt.show()

In [ ]:
plt.show()